[![Binder](https://mybinder.org/badge_logo.svg)](https://lab.mlpack.org/v2/gh/mlpack/examples/master?urlpath=lab%2Ftree%2Fq_learning%2Fbipedal_walker_sac.ipynb)

You can easily run this notebook at https://lab.mlpack.org/

Here, we train a [Soft Actor-Critic](https://arxiv.org/abs/1801.01290) agent to get high scores for the [Bipedal Walker](https://gym.openai.com/envs/BipedalWalker-v2/) environment. 

We make the agent train and test on OpenAI Gym toolkit's GUI interface provided through a distributed infrastructure (TCP API). More details can be found [here](https://github.com/zoq/gym_tcp_api).

A video of the trained agent can be seen in the end.

## Including necessary libraries and namespaces

In [1]:
#include <mlpack/core.hpp>

In [2]:
#include <mlpack/methods/ann/ffn.hpp>
#include <mlpack/methods/reinforcement_learning/sac.hpp>
#include <mlpack/methods/ann/loss_functions/empty_loss.hpp>
#include <mlpack/methods/ann/init_rules/gaussian_init.hpp>
#include <mlpack/methods/reinforcement_learning/environment/env_type.hpp>
#include <mlpack/methods/reinforcement_learning/training_config.hpp>

In [3]:
// Used to run the agent on gym's environment (provided externally) for testing.
#include <gym/environment.hpp>

In [4]:
// Used to generate and display a video of the trained agent.
#include "xwidgets/ximage.hpp"
#include "xwidgets/xvideo.hpp"
#include "xwidgets/xaudio.hpp"

In [5]:
using namespace mlpack;

In [6]:
using namespace mlpack::ann;

In [7]:
using namespace ens;

In [8]:
using namespace mlpack::rl;

## Initializing the agent

In [9]:
// Set up the state and action space.
ContinuousActionEnv::State::dimension = 24;
ContinuousActionEnv::Action::size = 4;

In [10]:
// Set up the actor and critic networks.
FFN<EmptyLoss<>, GaussianInitialization>
    policyNetwork(EmptyLoss<>(), GaussianInitialization(0, 0.1));
policyNetwork.Add(new Linear<>(ContinuousActionEnv::State::dimension, 64));
policyNetwork.Add(new ReLULayer<>());
policyNetwork.Add(new Linear<>(64, ContinuousActionEnv::Action::size));
policyNetwork.Add(new TanHLayer<>());

FFN<EmptyLoss<>, GaussianInitialization>
    qNetwork(EmptyLoss<>(), GaussianInitialization(0, 0.1));
qNetwork.Add(new Linear<>(ContinuousActionEnv::State::dimension +
                          ContinuousActionEnv::Action::size, 64));
qNetwork.Add(new ReLULayer<>());
qNetwork.Add(new Linear<>(64, 1));

In [11]:
// Set up the policy method.
RandomReplay<ContinuousActionEnv> replayMethod(32, 10000);

In [12]:
// Set up training configurations.
TrainingConfig config;
config.ExplorationSteps() = 3200;
config.TargetNetworkSyncInterval() = 1;
config.UpdateInterval() = 1;

In [13]:
// Set up Soft actor-critic agent.
SAC<ContinuousActionEnv, decltype(qNetwork), decltype(policyNetwork), AdamUpdate>
    agent(config, qNetwork, policyNetwork, replayMethod);

## Preparation for training the agent

In [14]:
// Set up the gym training environment.
gym::Environment env("gym.kurg.org", "4040", "BipedalWalker-v3");

// Initializing training variables.
std::vector<double> returnList;
size_t episodes = 0;
bool converged = true;

// The number of episode returns to keep track of.
size_t consecutiveEpisodes = 25;

In [15]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "BipedalWalker-v3");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {agent.Action().action};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();
std::cout << url;
auto video = xw::video_from_url(url).finalize();
video

 Total steps: 106	 Total reward: -93.3435
https://gym.kurg.org/a5eb46caf7934/output.webm

A Jupyter widget

In [16]:
// Function to train the agent on the Bipedal Walker gym environment.
void train(const size_t numSteps)
{
  agent.Deterministic() = false;
  std::cout << "Training for " << numSteps << " steps." << std::endl;
  while (agent.TotalSteps() < numSteps)
  {
    double episodeReturn = 0;
    env.reset();
    size_t steps = 0;
    do
    {
      agent.State().Data() = env.observation;
      agent.SelectAction();
      arma::mat action = {agent.Action().action};

      env.step(action);
      ContinuousActionEnv::State nextState;
      nextState.Data() = env.observation;

      replayMethod.Store(agent.State(), agent.Action(), env.reward, nextState,
          env.done, 0.99);
      episodeReturn += env.reward;
      agent.TotalSteps()++;
      steps++;
      if (agent.Deterministic() || agent.TotalSteps() < config.ExplorationSteps())
        continue;
      for (size_t i = 0; i < config.UpdateInterval(); i++)
        agent.Update();
    } while (!env.done);
    returnList.push_back(episodeReturn);
    episodes += 1;

    if (returnList.size() > consecutiveEpisodes)
      returnList.erase(returnList.begin());
        
    double averageReturn = std::accumulate(returnList.begin(),
                                           returnList.end(), 0.0) /
                           returnList.size();
    if(episodes % 1 == 0)
    {
      std::cout << "Avg return for " << returnList.size()
          << " episodes: " << averageReturn
          << "\t Episode return: " << episodeReturn
          << "\t steps: " << steps
          << "\t Total steps: " << agent.TotalSteps() << std::endl;
    }
  }
}

## Let the training begin

In [ ]:
// Training the agent for a total of at least 20000 steps.
train(20000)

Training for 20000 steps.
Avg return for 1 episodes: -22.4558	 Episode return: -22.4558	 steps: 1600	 Total steps: 1600
Avg return for 2 episodes: -71.2708	 Episode return: -120.086	 steps: 142	 Total steps: 1742
Avg return for 3 episodes: -78.7429	 Episode return: -93.6872	 steps: 87	 Total steps: 1829
Avg return for 4 episodes: -82.537	 Episode return: -93.9192	 steps: 90	 Total steps: 1919
Avg return for 5 episodes: -93.9891	 Episode return: -139.798	 steps: 1389	 Total steps: 3308
Avg return for 6 episodes: -98.9828	 Episode return: -123.951	 steps: 64	 Total steps: 3372
Avg return for 7 episodes: -102.657	 Episode return: -124.699	 steps: 78	 Total steps: 3450
Avg return for 8 episodes: -102.428	 Episode return: -100.828	 steps: 70	 Total steps: 3520
Avg return for 9 episodes: -102.054	 Episode return: -99.0604	 steps: 69	 Total steps: 3589
Avg return for 10 episodes: -102.289	 Episode return: -104.402	 steps: 74	 Total steps: 3663
Avg return for 11 episodes: -102.715	 Episode ret

## Testing the trained agent

In [ ]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "BipedalWalker-v3");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {agent.Action().action};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();
std::cout << url;
auto video = xw::video_from_url(url).finalize();
video

## A little more training...

In [ ]:
// Training the same agent for a total of at least 100000 steps.
train(100000)

# Final agent testing!

In [ ]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "BipedalWalker-v3");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action().action[0] * 2)};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();
std::cout << url;
auto video = xw::video_from_url(url).finalize();
video